In [7]:
import duckdb
from pipelines.tasks.config.common import DUCKDB_FILE

con = duckdb.connect(database=DUCKDB_FILE, read_only=True)
# show all tables in DB
con.sql("SHOW TABLES;").show()

┌──────────────────────────────────┐
│               name               │
│             varchar              │
├──────────────────────────────────┤
│ ana__resultats_communes          │
│ atlasante_udi                    │
│ cog_communes                     │
│ edc_communes                     │
│ edc_prelevements                 │
│ edc_resultats                    │
│ int__lien_cdreseau_refreneceprel │
│ int__lien_commune_cdreseau       │
│ int__mapping_category_simple     │
│ int__prelevements_uniques        │
│ int__resultats_udi_communes      │
│ laposte_communes                 │
│ mapping_categories               │
│ stg_communes__cog                │
│ stg_communes__laposte            │
│ stg_edc__communes                │
│ stg_edc__prevelevements          │
│ stg_edc__resultats               │
├──────────────────────────────────┤
│             18 rows              │
└──────────────────────────────────┘



In [5]:
# describe atlasante_udi table
df = con.sql("DESCRIBE atlasante_udi;").df()
print(df)

       column_name               column_type null   key default extra
0              gid                   INTEGER  YES  None    None  None
1         code_udi                   VARCHAR  YES  None    None  None
2         code_ins                   VARCHAR  YES  None    None  None
3          ins_nom                   VARCHAR  YES  None    None  None
4         ins_etat                   VARCHAR  YES  None    None  None
5          udi_pop                   VARCHAR  YES  None    None  None
6         uge_code                   VARCHAR  YES  None    None  None
7          uge_nom                   VARCHAR  YES  None    None  None
8       uge_exp_cn                   VARCHAR  YES  None    None  None
9      uge_exp_nom                   VARCHAR  YES  None    None  None
10       uge_mo_cn                   VARCHAR  YES  None    None  None
11      uge_mo_nom                   VARCHAR  YES  None    None  None
12            geom                  GEOMETRY  YES  None    None  None
13  ingestion_date  

In [7]:
# install spatial extention for spactial functions
con.sql("INSTALL spatial;")
# Load spatial extension
con.sql("LOAD spatial;")

In [8]:
# show same paris's UDI
df = con.sql("Select * from atlasante_udi where uge_nom like '%EAU DE PARIS%'").df()
df.head()

,gid,code_udi,code_ins,ins_nom,ins_etat,udi_pop,uge_code,uge_nom,uge_exp_cn,uge_exp_nom,uge_mo_cn,uge_mo_nom,geom,ingestion_date
0,1555,075000221,075000221,CENTRE,ACT,,0750001,EAU DE PARIS,07500034,EAU DE PARIS,07500034,EAU DE PARIS,"[5, 4, 0, 0, 0, 0, 0, 0, 172, 97, 17, 64, 237,...",2025-03-25 10:19:39.593000+01:00
1,2486,075000227,075000227,EST,ACT,,0750001,EAU DE PARIS,07500034,EAU DE PARIS,07500034,EAU DE PARIS,"[5, 4, 0, 0, 0, 0, 0, 0, 131, 3, 22, 64, 150, ...",2025-03-25 10:19:39.593000+01:00
2,9623,075000225,075000225,SUD OUEST,ACT,,0750001,EAU DE PARIS,07500034,EAU DE PARIS,07500034,EAU DE PARIS,"[5, 4, 0, 0, 0, 0, 0, 0, 155, 89, 14, 64, 66, ...",2025-03-25 10:19:39.593000+01:00
3,11249,075000222,075000222,NORD OUEST,ACT,,0750001,EAU DE PARIS,07500034,EAU DE PARIS,07500034,EAU DE PARIS,"[5, 4, 0, 0, 0, 0, 0, 0, 202, 109, 17, 64, 25,...",2025-03-25 10:19:39.593000+01:00


In [9]:
# Paris North (Nord - near Saint-Denis):
# latitudeN = 48.9358
# longitudeN = 2.3538
# Paris South
# latitudeS = 48.8186
# longitudeS = 2.3326
# Paris West (Ouest - near Porte Maillot/Neuilly):
# latitudeW = 48.8781
# longitudeW = 2.2785
# Central Paris (Centre - Notre-Dame):
latitude = 48.8566
longitude = 2.3522

sql = f"""
SELECT *
FROM atlasante_udi
WHERE ST_Contains(geom, ST_GeomFromText('POINT({longitude} {latitude})'));
"""
df = con.sql(sql).df()
df.head()

,gid,code_udi,code_ins,ins_nom,ins_etat,udi_pop,uge_code,uge_nom,uge_exp_cn,uge_exp_nom,uge_mo_cn,uge_mo_nom,geom,ingestion_date
0,1555,075000221,075000221,CENTRE,ACT,,0750001,EAU DE PARIS,07500034,EAU DE PARIS,07500034,EAU DE PARIS,"[5, 4, 0, 0, 0, 0, 0, 0, 172, 97, 17, 64, 237,...",2025-03-25 10:19:39.593000+01:00


In [14]:
con.close()